In [1]:
import json
import pandas as pd

In [2]:
data = pd.read_csv('../../data/result/train_with_suggestions_df_majority_vote_gpt4o_preferred.csv')

In [3]:
data = data[['text', 'majority_vote']]

In [4]:
data.head()

,text,majority_vote
0,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,needs enhancements
1,اقتراحاتي للتحسين: provide more new books,needs to be added
2,اقتراحاتي للتحسين: زيادة ساعات العملي,needs to be added
3,اقتراحاتي للإضافة: د. اعتماد من الدكاتره المحت...,needs to be removed
4,اقتراحاتي للإضافة: زيادة التمارين في المادة، ا...,needs to be added


In [5]:
augmentation_path = '../../data/data_augmentation/gpt_augmentation/final_augmentation_full_data_5_examples.json'

In [6]:
with open(augmentation_path) as f:
    augmentation_data = json.load(f)

In [7]:
data['majority_vote'].value_counts()

needs to be added      174
needs enhancements      86
none                    51
needs to be removed     14
Name: majority_vote, dtype: int64

In [8]:
# data['text'] = data['text'].apply(lambda x: [x])
# data.head()

In [9]:
len(list(augmentation_data.values())[0])

5

In [10]:
def augment_one_class(data, augmentation_data, class_name, labels_col, augmentation_col, target_length, duplicate=False):
    class_data = list(data[data[labels_col]==class_name][augmentation_col])
    
    if len(class_data) == 0:
        raise ValueError(f"Class '{class_name}' not found in column '{labels_col}'")
    
    augmented = []
    
    num_augmentation_examples = len(list(augmentation_data.values())[0])
    
    flag = False
    for i in range(num_augmentation_examples):
        for example in class_data:
            if len(class_data + augmented) >= target_length:
                flag = True
                break
            augmented.append(list(augmentation_data[example].values())[i])
        
        if flag:
            break
    
    if duplicate:
        if len(class_data + augmented) < target_length:
            for example in class_data:
                augmented.append(example)
                if len(class_data + augmented) >= target_length:
                    break
        
            
            for example in augmented.copy():
                if len(class_data + augmented) >= target_length:
                    break
                
                augmented.append(example)
                
    augmented_df = pd.DataFrame()
    augmented_df[augmentation_col] = augmented
    augmented_df[labels_col] = class_name
    
    return pd.concat([data, augmented_df], ignore_index=True)

In [11]:
target_length = 174
duplicate = False

In [12]:
for class_name in list(data['majority_vote'].unique()):
    data = augment_one_class(
        data=data, augmentation_data=augmentation_data, class_name=class_name, labels_col='majority_vote', 
        augmentation_col='text', target_length=target_length, duplicate=duplicate
    )

In [13]:
data['majority_vote'].value_counts()

needs enhancements     174
needs to be added      174
none                   174
needs to be removed     84
Name: majority_vote, dtype: int64

In [14]:
# for i, text_o in enumerate(data['text']):
#     data.at[i, 'text'] = text_o + list(augmentation_data[text_o[0]].values())

In [15]:
# data = data.explode('text', ignore_index=True)

In [16]:
data.head()

,text,majority_vote
0,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,needs enhancements
1,اقتراحاتي للتحسين: provide more new books,needs to be added
2,اقتراحاتي للتحسين: زيادة ساعات العملي,needs to be added
3,اقتراحاتي للإضافة: د. اعتماد من الدكاتره المحت...,needs to be removed
4,اقتراحاتي للإضافة: زيادة التمارين في المادة، ا...,needs to be added


In [17]:
len(data)

606

In [18]:
data.head()

,text,majority_vote
0,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,needs enhancements
1,اقتراحاتي للتحسين: provide more new books,needs to be added
2,اقتراحاتي للتحسين: زيادة ساعات العملي,needs to be added
3,اقتراحاتي للإضافة: د. اعتماد من الدكاتره المحت...,needs to be removed
4,اقتراحاتي للإضافة: زيادة التمارين في المادة، ا...,needs to be added


In [19]:
data['majority_vote'].value_counts()

needs enhancements     174
needs to be added      174
none                   174
needs to be removed     84
Name: majority_vote, dtype: int64

In [20]:
random_state=42

In [21]:
data = data.sample(frac=1.0, random_state=random_state)

In [22]:
data.to_csv('../../data/result/final_suggestion_train_augmented_gpt4o_5_examples1.csv', index=False)